In [1]:
import dspy

/home/student1/miniconda3/envs/llm_env_achal/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lm = dspy.LM('ollama_chat/llama3.1:8b', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [3]:
system_prompt = '''You will be given some tools and some context. Use those tool and context if needed to solve the task given to you. Do not do more than what is asked. Your answer must only contain the tool(s) you used and the arguments given to those functions in the form of a json. If no tool is used give a simple json with key as message and value as the answer.'''
task = '''Go to the market and buy 5 apples?'''
context = '''You are Bob and are currently at home.'''

In [4]:
def goto(location:str):
    '''
    Go to a location based on the input. Possible locations are "home", "market", "forest"
    
    Args:
    location (str): the location to go to
    '''
    
    if location in ["market", "forest", "home"]:
        return f"You are now in the the {location}"
    
    else:
        return f"Location {location} not found"

def pick_up(item:str):
    '''
    Pick up an item based on the input. Possible items are "apple", "shield", "potion"
    
    Args:
    item (str): the item to pick up
    '''
    
    if item in ["apple", "shield", "potion"]:
        return f"Picked up {item}"
    
    else:
        return f"Item {item} not found"

def talk_to_person(person:str, message:str):
    '''
    Speak message to a person based on the input. Possible people are "guard", "merchant", "wizard"
    
    Args:
    person (str): the person to speak to
    message (str): the message to speak
    '''
    if person in ["guard", "merchant", "wizard"]:
        return f"Spoke to {person} and said: {message}"
    
    else:
        return f"Person {person} not found"

def buy_item(item:str, quantity:int, price:int):
    '''
    Buy an item based on the input. Possible items are "apple", "shield", "potion"

    Args:
    item (str): the item to buy
    quantity (int): the quantity of the item to buy
    price (int): the price of the item
    '''
    if item in ["apple", "shield", "potion"]:
        return f"Bought {quantity} {item}(s) for {price} gold"
    
    else:
        return f"Item {item} not found"

def agent_memory():
    ''' Returns the agent's own memory
    
    Returns:
    str: the agent's memory
    '''
    return '''Your name is Bob and are currently at home.'''

tools = [goto, pick_up, buy_item, talk_to_person, agent_memory]

In [5]:
gen = dspy.ReAct('system, context, task -> answer', tools=tools, max_iters=1)

In [6]:
interaction = gen(system = system_prompt, task = task, context = context)

In [7]:
print("TRAJECTORY:\n", interaction['trajectory'])
print()
print("RATIONALE:\n",interaction['reasoning'])
print()
print("FINAL ANSWER:\n",interaction['answer'])

TRAJECTORY:
 {'thought_0': 'I need to go to the market first.', 'tool_name_0': 'goto', 'tool_args_0': {'location': 'market'}, 'observation_0': 'You are now in the the market'}

RATIONALE:
 I will use the "goto" tool to move to the market, and then I will buy 5 apples.

FINAL ANSWER:
 {
  "tools_used": [
    {
      "name": "goto",
      "args": {"location": "market"}
    }
  ],
  "message": "Buy 5 apples"
}
